<a href="https://colab.research.google.com/github/guanyu1127/Programming-Language/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

日常支出速算與分攤（作業一） 目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。 AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。 Sheet 欄位：date, category, item, amount, payer

In [1]:
# =========================================================
# HW1：日常支出速算與分攤（Google Sheet + Gradio）
# 條件對應：
# 1) I/O to GoogleSheet：append 新紀錄、讀取並寫回 Summary
# 2) 使用 if / else / for / def
# 3) Gradio 網頁介面
# =========================================================

# ========== 0) 安裝套件 ==========
!pip -q install gspread gspread-dataframe pandas google-auth google-auth-oauthlib pytz gradio

# ========== 1) 基本設定 ==========
SHEET_URL = "https://docs.google.com/spreadsheets/d/1zgd1GM9-Vi0JlwacPmbZF0Ci45QSX2t0EIZY1fmzfFE/edit?gid=0#gid=0"
DATA_SHEET_NAME = "消費紀錄"     # 請確認試算表存在此分頁，且第一列為：date, category, item, amount, payer
SUMMARY_SHEET_NAME = "Summary"   # 若不存在會自動建立
TIMEZONE = "Asia/Taipei"

# ========== 2) Google 驗證 ==========
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# ========== 3) 匯入依賴 ==========
import pandas as pd
import numpy as np
import pytz
from datetime import datetime, timedelta, date
import gradio as gr

# ========== 4) 基礎工具函式（def / if / else）==========
def open_spreadsheet():
    """開啟試算表；若網址錯誤會丟出例外。"""
    if "docs.google.com/spreadsheets/d/" not in SHEET_URL:
        raise ValueError("SHEET_URL 格式不正確。請貼上像 https://docs.google.com/spreadsheets/d/<ID>/edit 的網址。")
    return gc.open_by_url(SHEET_URL)

def get_ws(ss, title):
    """取得指定工作表；若不存在可選擇建立。"""
    try:
        return ss.worksheet(title)
    except gspread.WorksheetNotFound:
        return None

def ensure_summary_sheet(ss):
    ws = get_ws(ss, SUMMARY_SHEET_NAME)
    if ws is None:
        ws = ss.add_worksheet(title=SUMMARY_SHEET_NAME, rows=200, cols=20)
    return ws

def parse_date_safe(x):
    if pd.isna(x) or str(x).strip() == "":
        return pd.NaT
    try:
        return pd.to_datetime(x)
    except Exception:
        return pd.NaT

def to_number(v):
    s = str(v).replace(",", "").strip()
    try:
        return float(s)
    except Exception:
        return np.nan

# ========== 5) 商業邏輯：讀、算、寫（大量使用 def / if / for）==========
def load_data():
    """從『消費紀錄』讀資料並做基本清理。若找不到分頁或資料無效會回傳錯誤訊息。"""
    ss = open_spreadsheet()
    ws_data = get_ws(ss, DATA_SHEET_NAME)
    if ws_data is None:
        raise RuntimeError(f"找不到工作表『{DATA_SHEET_NAME}』。請先在試算表建立該分頁並放上欄位列。")

    values = ws_data.get_all_values()
    if not values:
        raise RuntimeError(f"『{DATA_SHEET_NAME}』是空的，請在第 1 列輸入：date, category, item, amount, payer")

    headers = [h.strip().lower() for h in values[0]]
    df = pd.DataFrame(values[1:], columns=headers)

    required = ["date", "category", "item", "amount", "payer"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise RuntimeError(f"缺少必要欄位：{missing}；請確認第 1 列為 {required}")

    # 轉型
    df["date"] = df["date"].apply(parse_date_safe)
    df["amount"] = df["amount"].apply(to_number)
    # 去除空列
    df = df.dropna(subset=["date","category","item","amount","payer"]).reset_index(drop=True)
    if df.empty:
        raise RuntimeError("有效資料為空。請確認每欄都有填、日期可解析、金額為數字。")
    return ss, df

def compute_summary(df):
    """計算總額、分類小計、AA 分攤、轉帳建議，以及週摘要。"""
    total_amount = float(df["amount"].sum())
    by_category = (
        df.groupby("category", dropna=False)["amount"]
          .sum().reset_index().sort_values("amount", ascending=False)
    )

    # AA
    members = sorted(df["payer"].dropna().astype(str).unique().tolist())
    n = len(members)
    equal_share = total_amount / n if n > 0 else 0.0
    paid_by = df.groupby("payer", dropna=False)["amount"].sum().reindex(members, fill_value=0).reset_index()
    paid_by.columns = ["payer", "paid"]
    aa_summary = paid_by.copy()
    aa_summary["equal_share"] = round(equal_share, 2)
    aa_summary["balance"] = (aa_summary["paid"] - equal_share).round(2)

    # 轉帳建議（for 迴圈/雙指標）
    pos = aa_summary[aa_summary["balance"] > 0][["payer","balance"]].copy().sort_values("balance", ascending=False).reset_index(drop=True)
    neg = aa_summary[aa_summary["balance"] < 0][["payer","balance"]].copy().sort_values("balance").reset_index(drop=True)
    transfers = []
    i, j = 0, 0
    while i < len(neg) and j < len(pos):
        owe_name, owe_amt = neg.loc[i,"payer"], -float(neg.loc[i,"balance"])
        rec_name, rec_amt = pos.loc[j,"payer"], float(pos.loc[j,"balance"])
        pay = round(min(owe_amt, rec_amt), 2)
        if pay > 0:
            transfers.append({"from": owe_name, "to": rec_name, "amount": pay})
            neg.loc[i,"balance"] = -(owe_amt - pay)
            pos.loc[j,"balance"] =  rec_amt - pay
        if neg.loc[i,"balance"] >= -1e-9: i += 1
        if pos.loc[j,"balance"] <=  1e-9: j += 1
    transfer_df = pd.DataFrame(transfers) if transfers else pd.DataFrame([{"from":"—","to":"—","amount":0}])

    # 週摘要（if/else）
    tz = pytz.timezone(TIMEZONE)
    today_local = datetime.now(tz).date()
    week_start = today_local - timedelta(days=today_local.weekday())
    week_end   = week_start + timedelta(days=6)
    dfw = df[(df["date"].dt.date >= week_start) & (df["date"].dt.date <= week_end)]
    if dfw.empty:
        weekly_text = f"本週（{week_start}~{week_end}）尚無消費紀錄。"
    else:
        top_cat = dfw.groupby("category")["amount"].sum().sort_values(ascending=False)
        tip = ""
        if not top_cat.empty:
            tip = f"最高支出類別「{top_cat.index[0]}」占比 {float(top_cat.iloc[0]/top_cat.sum()*100):.1f}%。"
        high = dfw.sort_values("amount", ascending=False).head(1)
        extra = ""
        if not high.empty and float(high.iloc[0]["amount"])>=1000:
            extra = f" 最大單筆「{high.iloc[0]['item']}」{int(high.iloc[0]['amount'])}。"
        weekly_text = f"本週（{week_start}~{week_end}）總支出 {float(dfw['amount'].sum()):.0f}。 {tip}{extra}"

    # 組 Summary 區塊（轉成原生型別）
    overview_rows = [
        ["指標","數值"],
        ["總支出", round(total_amount,2)],
        ["成員（AA）", ", ".join(members)],
        ["平均分攤(每人)", round(equal_share,2)],
        ["週期(本週)", f"{week_start} ~ {week_end}"],
    ]
    cat_table = [["category","subtotal"]] + [[str(r["category"]), round(float(r["amount"]),2)] for _,r in by_category.iterrows()]
    aa_table  = [["payer","paid","equal_share","balance"]] + [
        [str(r["payer"]), round(float(r["paid"]),2), round(float(r["equal_share"]),2), round(float(r["balance"]),2)]
        for _,r in aa_summary.iterrows()
    ]
    transfer_table = [["from","to","amount"]] + [[str(r["from"]), str(r["to"]), round(float(r["amount"]),2)] for _,r in transfer_df.iterrows()]
    summary_paragraph = [["Weekly Summary"], [weekly_text]]

    return {
        "overview_rows": overview_rows,
        "cat_table": cat_table,
        "aa_table": aa_table,
        "transfer_table": transfer_table,
        "weekly_text": weekly_text,
        "df": df,
        "by_category": by_category,
        "aa_summary": aa_summary,
        "transfer_df": transfer_df
    }

def write_summary(blocks):
    """把計算結果寫回 Summary 分頁（Google Sheet Output）。"""
    ss = open_spreadsheet()
    ws = ensure_summary_sheet(ss)

    def _to_native(v):
        # NaN -> 空字串
        try:
            if pd.isna(v):
                return ""
        except Exception:
            pass
        if isinstance(v, (np.integer,)): return int(v)
        if isinstance(v, (np.floating,)): return float(v)
        if isinstance(v, (pd.Timestamp, datetime, date)):
            return pd.to_datetime(v).strftime("%Y-%m-%d")
        return str(v)

    def write_block(start_row, start_col, values):
        conv = [[_to_native(x) for x in row] for row in values]
        end_row = start_row + len(conv) - 1
        end_col = start_col + len(conv[0]) - 1
        rng = gspread.utils.rowcol_to_a1(start_row, start_col) + ":" + gspread.utils.rowcol_to_a1(end_row, end_col)
        ws.update(values=conv, range_name=rng)
        return end_row + 2

    ws.clear()
    row = 1
    row = write_block(row, 1, [["=== Overview ==="]]);            row = write_block(row, 1, blocks["overview_rows"])
    row = write_block(row, 1, [["=== Category Subtotals ==="]]);  row = write_block(row, 1, blocks["cat_table"])
    row = write_block(row, 1, [["=== AA Split ==="]]);            row = write_block(row, 1, blocks["aa_table"])
    row = write_block(row, 1, [["=== Suggested Transfers ==="]]); row = write_block(row, 1, blocks["transfer_table"])
    row = write_block(row, 1, [["=== Weekly Summary (Auto) ==="]]); row = write_block(row, 1, [["Weekly Summary"], [blocks["weekly_text"]]])
    return "已寫回 Summary ✅"

def append_record(record):
    """
    新增一筆消費紀錄到『消費紀錄』（Google Sheet Input）
    record = (date_str, category, item, amount, payer)
    """
    (date_str, category, item, amount, payer) = record
    # 基本驗證（if/else）
    if not all([date_str, category, item, amount, payer]):
        return False, "請把欄位填完整再送出。"
    try:
        dt = pd.to_datetime(date_str)
    except Exception:
        return False, "日期格式無法解析，建議使用 YYYY-MM-DD。"
    try:
        amt = float(str(amount).replace(",","").strip())
    except Exception:
        return False, "金額需為數字。"

    ss = open_spreadsheet()
    ws = get_ws(ss, DATA_SHEET_NAME)
    if ws is None:
        # 若資料分頁不存在，就先建立並補上標題列
        ws = ss.add_worksheet(title=DATA_SHEET_NAME, rows=200, cols=10)
        ws.update(values=[["date","category","item","amount","payer"]], range_name="A1:E1")

    # 追加一列（Append Row）
    ws.append_row([dt.strftime("%Y-%m-%d"), category, item, amt, payer])
    return True, "已新增一筆紀錄 ✅"

# ========== 6) Gradio 介面 ==========
with gr.Blocks(title="日常支出速算與分攤") as demo:
    gr.Markdown("## 🧮 日常支出速算與分攤（Google Sheet + Gradio）")

    with gr.Tab("① 新增一筆消費（寫入 Google Sheet）"):
        date_in   = gr.Textbox(label="date (YYYY-MM-DD)")
        cat_in    = gr.Textbox(label="category（例如：餐飲/交通/娛樂）")
        item_in   = gr.Textbox(label="item")
        amt_in    = gr.Textbox(label="amount（數字）")
        payer_in  = gr.Textbox(label="payer（付款人）")
        add_btn   = gr.Button("送出")
        add_msg   = gr.Markdown()

        def on_add(date_in, cat_in, item_in, amt_in, payer_in):
            ok, msg = append_record((date_in, cat_in, item_in, amt_in, payer_in))
            return msg

        add_btn.click(on_add, inputs=[date_in, cat_in, item_in, amt_in, payer_in], outputs=add_msg)

    with gr.Tab("② 重算並寫回 Summary（讀→算→寫）"):
        run_btn   = gr.Button("執行")
        out_msg   = gr.Markdown()
        df_view   = gr.Dataframe(label="消費紀錄（預覽）")
        cat_view  = gr.Dataframe(label="分類小計")
        aa_view   = gr.Dataframe(label="AA 分攤")
        tr_view   = gr.Dataframe(label="轉帳建議")
        weekly    = gr.Markdown(label="週摘要")

        def on_run():
            try:
                ss, df = load_data()
                blocks = compute_summary(df)
                msg = write_summary(blocks)
                return (msg, df, blocks["by_category"], blocks["aa_summary"], blocks["transfer_df"], f"**{blocks['weekly_text']}**")
            except Exception as e:
                return (f"❌ 錯誤：{e}", pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), "")
        run_btn.click(on_run, inputs=None, outputs=[out_msg, df_view, cat_view, aa_view, tr_view, weekly])

    with gr.Tab("③ 只看週摘要（不寫回）"):
        look_btn = gr.Button("計算週摘要")
        show_md  = gr.Markdown()

        def on_peek():
            try:
                _, df = load_data()
                blocks = compute_summary(df)
                return f"**{blocks['weekly_text']}**"
            except Exception as e:
                return f"❌ 錯誤：{e}"
        look_btn.click(on_peek, inputs=None, outputs=show_md)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://71f92d6e5889861475.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
